In [ ]:
!pip install hvplot holoviews datashader geoviews datashader tarfile tqdm xarray radolan_to_netcdf numpy==2.0 cartopy jupyter_bokeh


In [1]:
import tarfile
import hvplot.pandas
import holoviews
import datashader
import geoviews
import holoviews as hv
hv.extension('bokeh', 'matplotlib')  # Oder je nachdem, welches Backend du nutzt
# import datashader
# import geoviews
# import cartopy
import tqdm
import xarray as xr
import hvplot.xarray
import numpy
import radolan_to_netcdf as rtn

# Download RADKLIM-YW data for May 2016

We want to look at the data for the flooding in the city of Braunsbach on 29th of May 2016.

**Note that RADKLIM data is corrected based on long-term statistics. Hence, these correction are not adjusted for individual events, like the one shown here. Like all radar rainfall estimate, and all rainfall estimates in general, the absolute values have to be treated with caution. The data is shown here only to highlight how easy it now is to access and explore RADOLAN and RADKLIM data.**

In [ ]:
!curl -O https://opendata.dwd.de/climate_environment/CDC/grids_germany/5_minutes/radolan/reproc/2017_002/bin/2016/YW2017.002_201605.tar

# Read data from nested tar file

Data is provided as monthly tar files, which contains dailt tar.gz files, which contain the 5-minute binary files. To avoid extracting everything first we use the nested loop-construct below and extract only the data we want on the fly.

In [2]:
fn_netcdf = './radklim-yw.nc'

In [ ]:
rtn.create_empty_netcdf(fn=fn_netcdf, product_name='YW')

In [ ]:

with tarfile.open('./data/YW2017.002_201605.tar', 'r') as tar:
    fn_list = tar.getnames()
    fn_list.sort()
    for fn in fn_list[28:29]:
        print(f'Extracting daily file {fn}')
        f = tar.extractfile(fn)
        with tarfile.open(fileobj=f) as tar_inner:
            fn_list_inner = tar_inner.getnames()
            fn_list_inner.sort()
            # We only extract the 5-minute data from the afternoon
            for fn_inner in tqdm.tqdm(fn_list_inner[-120:-60]):
                #print(f'parsing {fn_inner}')
                data, metadata = rtn.read_in_one_bin_file(tar_inner.extractfile(fn_inner))
                rtn.append_to_netcdf(
                    fn_netcdf, 
                    data_list=[data, ], 
                    metadata_list=[metadata, ],
                )

In [3]:
ds = xr.open_dataset('radklim-yw.nc')
ds

<xarray.Dataset> Size: 1GB
Dimensions:          (x: 900, y: 1100, time: 60)
Coordinates:
  * x                (x) float64 7kB -443.5 -442.5 -441.5 ... 453.5 454.5 455.5
  * y                (y) float64 9kB -4.759e+03 -4.758e+03 ... -3.66e+03
    latitudes        (y, x) float64 8MB ...
    longitudes       (y, x) float64 8MB ...
  * time             (time) datetime64[ns] 480B 2016-05-29T13:59:59.999999936...
Data variables:
    rainfall_amount  (time, y, x) float64 475MB ...
    maxrange         (time) int16 120B ...
    radarlocations   (time) <U63 15kB ...
    secondary        (time, y, x) float32 238MB ...
    nodatamask       (time, y, x) float32 238MB ...
    cluttermask      (time, y, x) float32 238MB ...
    radolan_grid     float64 8B ...
Attributes:
    title:        RADOLAN YW rainfall data
    producttype:  YW
    institution:  Deutscher Wetterdienst (DWD)
    history:      Created at 2024-10-25 12:16:59.525435
    Conventions:  CF-1.6

# Plot 5-minute rainfall maps with time slider 

Note that I have not yet succeeded in adding a marker for the locations of Braunsbach in this plot. See the WIP version at the end of this notebook.

In [4]:
rainfall_map = ds.rainfall_amount.hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,  # Setze dies vorübergehend auf False, um zu testen 
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 10), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map

# from IPython.display import display
# display(rainfall_map)


BokehModel(combine_events=True, render_bundle={'docs_json': {'87a24c31-2ced-4ff0-9c57-19f29de5a4ce': {'version…

# Plot rainfall accumulation over the period covered by the data

Add coordinates of the village of Braunsbach as `pandas.Dataframe`, because this currently seems to be the easiest way to add a marker to the `hvplot` map.

In [5]:
import pandas as pd
import hvplot.pandas

braunsbach = pd.DataFrame(data={'x': [9.7913, ], 'y': [49.1992, ]})

In [6]:
print('First time stamp in data: ' + str(ds.rainfall_amount.time.values[0]))
print('Last time stamp in data : ' + str(ds.rainfall_amount.time.values[-1]))

First time stamp in data: 2016-05-29T13:59:59.999999936
Last time stamp in data : 2016-05-29T18:55:00.000000064


In [7]:
rainfall_map = ds.rainfall_amount.sum(dim='time').hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 100), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map * braunsbach.hvplot.points(x='x', y='y', geo=True, color='black')

BokehModel(combine_events=True, render_bundle={'docs_json': {'e9abc2b8-daee-4eac-9691-49cc24d00710': {'version…

# [WIP] Rainfall map with time slider and marker for Braunsbach

Unfortunately the addition of `hvplot.points` to the map results in resizing the plot when the time slider is dragged. I did not yet find out how to solve this...

In [8]:
rainfall_map = ds.rainfall_amount.hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 10), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map * braunsbach.hvplot.points(x='x', y='y', geo=True, project=True, color='black')

BokehModel(combine_events=True, render_bundle={'docs_json': {'1fabec9a-9f6d-4daf-8705-ae8ea2e3b865': {'version…